In [2]:
import requests
import json
from config import n_key
from domains import domains
import os
import pandas as pd
#run pip install nltk in your terminal with your PythonData environment activated if you have not already installed nltk
import nltk 
# nltk.download('vader_lexicon') <- you will need to run this the first time you run this code 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
SID = SentimentIntensityAnalyzer()

In [3]:
# change these variables to your desired search and your initials 
keyword = 'immigrants'
initials = 'RCK_2'

In [4]:
# n_key is your API key for NewsAPI
# keyword is the word you are interested in searching
# source_list is the list of US news sources available in NewsAPI
# initials is a string of your initials

def getNewsAPIData(n_key, keyword, domains, initials):
    base_url = ('http://newsapi.org/v2/everything?')
    params = {
        'language': 'en',
        'pageSize': 100,
        'sortBy': 'relevance',
        'apiKey': n_key,
        'q': keyword,    
    }
    
    data_master = []
    
    for domain in domains:
        params['domains'] = domain
        response = requests.get(base_url, params)
        data = response.json()


        articles = data['articles']

        for index in range(0, len(articles)): 
            article_dict = {
                'Keyword': keyword,
                'Source': articles[index]['source']['name'],
                'Author': articles[index]['author'],
                'Title': articles[index]['title'],
                'URL': articles[index]['url'],
                'Text': articles[index]['content'],
                'Published': articles[index]['publishedAt']}
            
            data_master.append(article_dict)

    data_df = pd.DataFrame(data_master)
    data_df.to_csv(f'initial{keyword}NewsAPIdata{initials}.csv', index=False)

    data_df['compound score'] = data_df['Title'].apply(lambda title: SID.polarity_scores(title)['compound'])
    data_df['negative score'] = data_df['Title'].apply(lambda title: SID.polarity_scores(title)['neg'])
    data_df['positive score'] = data_df['Title'].apply(lambda title: SID.polarity_scores(title)['pos'])
    data_df['neutral score'] = data_df['Title'].apply(lambda title: SID.polarity_scores(title)['neu'])

    data_df.to_csv(f'sentiment{keyword}NewsAPIdata{initials}.csv', index=False)


In [4]:
getNewsAPIData(n_key, keyword, domains, initials)

In [5]:
sentiment_data = pd.read_csv(f'sentiment{keyword}NewsAPIdata{initials}.csv')
sentiment_data.head()

,Keyword,Source,Author,Title,URL,Text,Published,compound score,negative score,positive score,neutral score
0,immigrants,The Washington Post,Nick Miroff,"Trump cuts refugee cap to lowest level ever, d...",https://www.washingtonpost.com/immigration/tru...,"The 15,000 figure, the lowest since the 1980 R...",2020-10-01T18:07:00Z,-0.8779,0.410,0.000,0.590
1,immigrants,The Washington Post,"Nick Miroff, Devlin Barrett",ICE preparing targeted arrests in ‘sanctuary c...,https://www.washingtonpost.com/immigration/tru...,"Chad Wolf, acting secretary of the Department ...",2020-09-29T22:37:00Z,-0.4404,0.182,0.000,0.818
2,immigrants,The Washington Post,"James A. McCann, Michael Jones-Correa",Trump’s attacks prompted Latino immigrants to ...,https://www.washingtonpost.com/politics/2020/1...,"How has all this affected Latino immigrants, t...",2020-10-01T11:00:12Z,-0.4404,0.195,0.000,0.805
3,immigrants,The Washington Post,Tim Elfrink,‘I have never supported Donald Trump’: Portlan...,https://www.washingtonpost.com/nation/2020/09/...,"Not so, says the sheriff in the Portland, Ore....",2020-09-30T07:28:00Z,0.0869,0.088,0.103,0.809
4,immigrants,The Washington Post,Tim Elfrink,‘This is a nightmare’: New Yorkers get absente...,https://www.washingtonpost.com/nation/2020/09/...,Malhotra wasnt alone. Numerous New Yorkers rep...,2020-09-29T10:57:00Z,-0.6369,0.271,0.000,0.729
